### 1. setting and data

In [ ]:
# install module
!pip install --upgrade pip
!pip install pyyaml pandas numpy matplotlib seaborn scikit-learn opencv-python
!pip iuninstall keras -y
!pip install tensorflow>=2.16


In [ ]:
# import module
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from config.load_config import load_config
from data.split_dataset import data_count, extract_balanced_dataset
from data.data_distribution import class_distribution
from data.dataloader import get_generators
from models.build_model import get_model
from models.build_model import build_model
from train.optimizer import get_optimizer
from train.callbacks import get_callbacks
from train.trainer import train_model
from utils.evaluation import (
    evaluate_model,
    plot_confusion_matrix, 
    plot_train_history, 
    show_top_misclassified, 
    plot_metrics_text,
    plot_cross_val_metrics
)
from utils.save_results import save_results
from train.cross_validation import run_cross_validation

In [ ]:
# Load configuration
config = load_config('/Users/vnlt/PROJECT/ex-cnn-cv/config/config_C.yaml')
data_dir = f"{config['base_output_dir']}/seed{config['seed']}"
batch_size = config['batch_size']
save_dir = f"results/{config['experiment_id']}"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
# check if config is loaded correctly
print(type(config))
print(config)

In [ ]:
import tensorflow as tf

print("🔎 TensorFlow 버전:", tf.__version__)
print("🔎 CUDA 지원 여부:", tf.test.is_built_with_cuda())

# 현재 인식된 GPU 목록
gpus = tf.config.list_physical_devices('GPU')
print("🔎 발견된 GPU:", gpus)

# 간단한 연산을 GPU에서 실행해보기
if gpus:
    try:
        with tf.device('/GPU:0'):
            a = tf.random.uniform((1000, 1000))
            b = tf.random.uniform((1000, 1000))
            c = tf.matmul(a, b)
        print("✅ GPU에서 matmul 연산 성공")
    except RuntimeError as e:
        print("⚠️ GPU 연산 실패:", e)
else:
    print("❌ GPU를 찾을 수 없습니다. CPU로만 동작합니다.")


#### select experimet type

In [ ]:
if config.get('use_cross_validation', False):
    print("✅ 교차 검증 모드를 실행합니다.")
    results = run_cross_validation(config) 
    # 최종 결과 시각화
    results_df = pd.DataFrame(results['individual_folds'])
    save_dir = f"results/{config['experiment_id']}" # 저장 경로 확인
    plot_cross_val_metrics(
        results_df, 
        config['experiment_id'], 
        save_path=f"{save_dir}/cross_val_metrics_boxplot.png"
    )
    
else:
    print("✅ 단일 훈련 모드를 실행합니다.")
    
    # 데이터 불러오기
    class_image_counts, images, min_count = data_count(
        original_dataset_dir=config['original_dataset_dir']
    )
    extract_balanced_dataset(
        original_dataset_dir=config['original_dataset_dir'],
        min_count=min_count,
        base_output_dir=f"{config['base_output_dir']}/seed{config['seed']}",
        seed=config['seed']
    )

    train_gen, val_gen, test_gen = get_generators(
        model_name=config['backbone_name'],
        input_shape=tuple(config['input_shape']),
        batch_size=config['batch_size'],
        data_dir=f"{config['base_output_dir']}/seed{config['seed']}",
        train_ratio=config['train_ratio'],
        val_ratio=config['val_ratio'],
        test_ratio=config['test_ratio'],
        seed=config['seed'],
        augmentations=config['augmentations']
    )
    
    class_distribution(
        train_gen,
        val_gen,
        test_gen,
        config,
        save_path=f"{save_dir}/dist_table.png"
    )
    print("data generators completed.")
    
    # 모델 빌드 및 학습
    model = build_model(
        backbone_name=config['backbone_name'],
        input_shape=tuple(config['input_shape']),
        num_classes=config['num_classes'],
        dropout_rate=config['dropout_rate']
    )
    optimizer = get_optimizer(
        optimizer_name=config['optimizer'],
        learning_rate=config['learning_rate'],
        weight_decay=config.get('weight_decay', 0.0)
    )
    callbacks = get_callbacks(
        model_name=config['backbone_name'],
        save_dir=f"results/{config['experiment_id']}",
        patience=config['patience']
    )
    print("model ready.")
    
    history = train_model(
        model,
        train_gen,
        val_gen,
        epochs=config['epochs'],
        callbacks=callbacks,
        optimizer=optimizer
    )
    print("training completed.")
    
    # 모델 평가
    y_true, y_pred, y_prob = evaluate_model(model, test_gen)
    class_names = list(test_gen.class_indices.keys())
    cm = confusion_matrix(y_true, y_pred)
    report = classification_report(y_true, y_pred, output_dict=True)
    print("evaluation completed.")
    
    # 결과 시각화 및 저장
    metrics_dict = {
        "accuracy": report["accuracy"],
        "precision": report["macro avg"]["precision"],
        "recall": report["macro avg"]["recall"],
        "f1_score": report["macro avg"]["f1-score"]
    }
    plot_metrics_text(
        metrics_dict=metrics_dict,
        experiment_id=config['experiment_id'],
        save_path=f"{save_dir}/metrics_text.png"
    )
    plot_confusion_matrix(
        cm, 
        class_names, 
        title="Confusion Matrix", 
        save_path=f"{save_dir}/confusion_matrix.png"
    )
    plot_train_history(
        history,
        save_path=f"{save_dir}/history_graph.png"
    )
    show_top_misclassified(
        y_true,
        y_pred,
        y_prob,
        class_names,
        generator=test_gen,
        model_name=config['experiment_id'],
        save_dir=save_dir,
        top_n=3
    )
    save_results(
        model_name=config['backbone_name'],
        history=history,
        cm=confusion_matrix(y_true, y_pred),
        class_names=list(test_gen.class_indices.keys()),
        report=classification_report(y_true, y_pred, target_names=list(test_gen.class_indices.keys()), output_dict=True),
        save_dir= save_dir
    )

    print(f"✅ 저장 완료: {save_dir}")